# 🤖 RAG Implementation с Cynosure Bridge

Полный пример реализации RAG (Retrieval Augmented Generation) системы.

## 📚 Установка зависимостей

In [ ]:
!pip install openai numpy faiss-cpu scikit-learn langchain chromadb

## 🚀 Базовая настройка

In [ ]:
import openai
import numpy as np
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
import json
from typing import List, Tuple

# Настройка клиента
client = openai.OpenAI(
    base_url="http://192.168.1.196:3000/v1",
    api_key="dummy-key"
)

print("✅ Настройка завершена")

## 🗄️ Создание базы знаний

In [ ]:
# Пример документов для базы знаний
knowledge_base = [
    {
        "id": 1,
        "title": "Что такое Cynosure Bridge",
        "content": "Cynosure Bridge - это OpenAI-совместимый прокси сервер, который позволяет использовать Claude MAX через стандартный OpenAI API. Он работает на порту 3000 и поддерживает все основные функции включая streaming и embeddings."
    },
    {
        "id": 2,
        "title": "Настройка и использование",
        "content": "Для использования Cynosure Bridge нужно просто поменять base_url в вашем OpenAI клиенте на http://192.168.1.196:3000/v1. API ключ может быть любым, так как используется Claude MAX подписка."
    },
    {
        "id": 3,
        "title": "Поддерживаемые модели",
        "content": "Bridge поддерживает все популярные OpenAI модели с автоматическим мапингом на Claude модели: gpt-4 -> claude-3-5-sonnet-20241022, gpt-4-turbo -> claude-3-5-sonnet-20241022, gpt-3.5-turbo -> claude-3-haiku-20240307."
    },
    {
        "id": 4,
        "title": "Векторные embeddings",
        "content": "Cynosure поддерживает создание embeddings через /v1/embeddings endpoint. Поддерживаются модели text-embedding-3-small, text-embedding-3-large и text-embedding-ada-002 с размерностями 1536 и 3072."
    },
    {
        "id": 5,
        "title": "Streaming и real-time",
        "content": "Bridge полностью поддерживает streaming через Server-Sent Events (SSE). Можно использовать параметр stream=true в запросах к /v1/chat/completions для получения ответов в реальном времени."
    }
]

print(f"📚 Загружено {len(knowledge_base)} документов")
for doc in knowledge_base:
    print(f"  - {doc['title']}")

## 🔍 Создание векторного индекса

In [ ]:
class VectorStore:
    def __init__(self):
        self.documents = []
        self.embeddings = []
        self.index = None
        
    def add_documents(self, docs: List[dict]):
        """Добавляет документы и создает их векторные представления"""
        print("🔄 Создание embeddings...")
        
        for i, doc in enumerate(docs):
            # Объединяем title и content для лучшего поиска
            text = f"{doc['title']}. {doc['content']}"
            
            # Создаем embedding
            response = client.embeddings.create(
                model="text-embedding-3-small",
                input=text
            )
            
            embedding = response.data[0].embedding
            
            self.documents.append(doc)
            self.embeddings.append(embedding)
            
            print(f"  ✅ {i+1}/{len(docs)}: {doc['title']}")
        
        # Создаем FAISS индекс
        dimension = len(self.embeddings[0])
        self.index = faiss.IndexFlatIP(dimension)  # Inner Product для косинусного сходства
        
        embeddings_array = np.array(self.embeddings).astype('float32')
        # Нормализуем векторы для косинусного сходства
        faiss.normalize_L2(embeddings_array)
        
        self.index.add(embeddings_array)
        
        print(f"🎯 Индекс создан: {self.index.ntotal} документов")
    
    def search(self, query: str, k: int = 3) -> List[Tuple[dict, float]]:
        """Поиск наиболее релевантных документов"""
        # Создаем embedding для запроса
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=query
        )
        
        query_embedding = np.array([response.data[0].embedding]).astype('float32')
        faiss.normalize_L2(query_embedding)
        
        # Поиск в индексе
        scores, indices = self.index.search(query_embedding, k)
        
        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx < len(self.documents):
                results.append((self.documents[idx], float(score)))
        
        return results

# Создаем и заполняем векторное хранилище
vector_store = VectorStore()
vector_store.add_documents(knowledge_base)

## 🤖 RAG Система

In [ ]:
class RAGSystem:
    def __init__(self, vector_store: VectorStore):
        self.vector_store = vector_store
        self.conversation_history = []
    
    def search_relevant_docs(self, query: str, k: int = 2) -> List[dict]:
        """Поиск релевантных документов"""
        results = self.vector_store.search(query, k)
        
        print(f"🔍 Найдено {len(results)} релевантных документов:")
        for doc, score in results:
            print(f"  📄 {doc['title']} (relevance: {score:.3f})")
        
        return [doc for doc, score in results]
    
    def build_context(self, relevant_docs: List[dict]) -> str:
        """Формирует контекст из релевантных документов"""
        if not relevant_docs:
            return "Контекст не найден."
        
        context_parts = []
        for doc in relevant_docs:
            context_parts.append(f"Документ: {doc['title']}\nСодержание: {doc['content']}")
        
        return "\n\n".join(context_parts)
    
    def generate_response(self, query: str, context: str) -> str:
        """Генерирует ответ на основе контекста"""
        system_prompt = """Ты - AI ассистент, который отвечает на вопросы на основе предоставленного контекста.
        
Правила:
1. Используй только информацию из предоставленного контекста
2. Если информации недостаточно, честно об этом скажи
3. Отвечай на русском языке
4. Будь конкретным и полезным
5. Если вопрос не связан с контекстом, вежливо объясни это

Контекст:
{context}""".format(context=context)
        
        messages = [
            {"role": "system", "content": system_prompt},
            *self.conversation_history,
            {"role": "user", "content": query}
        ]
        
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            max_tokens=800,
            temperature=0.1  # Низкая температура для более точных ответов
        )
        
        return response.choices[0].message.content
    
    def ask(self, query: str, include_sources: bool = True) -> dict:
        """Основной метод для вопросов к RAG системе"""
        print(f"❓ Вопрос: {query}")
        print("=" * 50)
        
        # Поиск релевантных документов
        relevant_docs = self.search_relevant_docs(query)
        
        # Формирование контекста
        context = self.build_context(relevant_docs)
        
        # Генерация ответа
        answer = self.generate_response(query, context)
        
        # Добавляем в историю разговора
        self.conversation_history.append({"role": "user", "content": query})
        self.conversation_history.append({"role": "assistant", "content": answer})
        
        # Ограничиваем историю
        if len(self.conversation_history) > 10:
            self.conversation_history = self.conversation_history[-10:]
        
        result = {
            "query": query,
            "answer": answer,
            "sources": relevant_docs if include_sources else None,
            "context_used": len(relevant_docs) > 0
        }
        
        return result
    
    def clear_history(self):
        """Очищает историю разговора"""
        self.conversation_history = []
        print("🧹 История разговора очищена")

# Создаем RAG систему
rag = RAGSystem(vector_store)
print("🚀 RAG система готова к работе!")

## 🧪 Тестирование RAG системы

In [ ]:
# Тест 1: Основной вопрос о продукте
result1 = rag.ask("Что такое Cynosure Bridge и как он работает?")
print(f"\n💬 Ответ: {result1['answer']}")
print(f"📚 Использованы источники: {[doc['title'] for doc in result1['sources']]}")

In [ ]:
# Тест 2: Технический вопрос
result2 = rag.ask("Какие модели поддерживает Bridge и как они мапятся?")
print(f"\n💬 Ответ: {result2['answer']}")
print(f"📚 Использованы источники: {[doc['title'] for doc in result2['sources']]}")

In [ ]:
# Тест 3: Вопрос о настройке
result3 = rag.ask("Как настроить и начать использовать Bridge?")
print(f"\n💬 Ответ: {result3['answer']}")
print(f"📚 Использованы источники: {[doc['title'] for doc in result3['sources']]}")

In [ ]:
# Тест 4: Вопрос вне контекста
result4 = rag.ask("Как приготовить борщ?")
print(f"\n💬 Ответ: {result4['answer']}")
print(f"📚 Использованы источники: {[doc['title'] for doc in result4['sources']] if result4['sources'] else 'Нет'}")

## 🔄 Интерактивный чат

In [ ]:
def interactive_chat():
    """Интерактивный чат с RAG системой"""
    print("🤖 RAG Ассистент готов отвечать на вопросы о Cynosure Bridge!")
    print("Введите 'quit' для выхода, 'clear' для очистки истории\n")
    
    while True:
        try:
            user_input = input("👤 Вопрос: ").strip()
            
            if user_input.lower() == 'quit':
                print("👋 До свидания!")
                break
            elif user_input.lower() == 'clear':
                rag.clear_history()
                continue
            elif not user_input:
                continue
            
            result = rag.ask(user_input, include_sources=False)
            print(f"\n🤖 {result['answer']}\n")
            print("-" * 50)
            
        except KeyboardInterrupt:
            print("\n👋 До свидания!")
            break
        except Exception as e:
            print(f"❌ Ошибка: {e}")

# Раскомментируйте для запуска интерактивного чата
# interactive_chat()

## 📊 Анализ производительности

In [ ]:
import time
from collections import defaultdict

class RAGAnalytics:
    def __init__(self):
        self.metrics = defaultdict(list)
    
    def benchmark_search(self, rag_system: RAGSystem, queries: List[str]):
        """Бенчмарк производительности поиска"""
        print("📊 Анализ производительности RAG системы...\n")
        
        for i, query in enumerate(queries, 1):
            print(f"🧪 Тест {i}/{len(queries)}: {query[:50]}...")
            
            start_time = time.time()
            result = rag_system.ask(query, include_sources=True)
            end_time = time.time()
            
            response_time = end_time - start_time
            sources_count = len(result['sources']) if result['sources'] else 0
            answer_length = len(result['answer'])
            
            self.metrics['response_times'].append(response_time)
            self.metrics['sources_found'].append(sources_count)
            self.metrics['answer_lengths'].append(answer_length)
            
            print(f"  ⏱️  Время ответа: {response_time:.2f}с")
            print(f"  📚 Источников: {sources_count}")
            print(f"  📝 Длина ответа: {answer_length} символов\n")
        
        self.print_summary()
    
    def print_summary(self):
        """Выводит сводную статистику"""
        if not self.metrics['response_times']:
            return
        
        response_times = self.metrics['response_times']
        sources_found = self.metrics['sources_found']
        answer_lengths = self.metrics['answer_lengths']
        
        print("📈 СВОДНАЯ СТАТИСТИКА")
        print("=" * 40)
        print(f"Общее количество запросов: {len(response_times)}")
        print(f"Среднее время ответа: {np.mean(response_times):.2f}с")
        print(f"Медианное время ответа: {np.median(response_times):.2f}с")
        print(f"Максимальное время: {max(response_times):.2f}с")
        print(f"Минимальное время: {min(response_times):.2f}с")
        print(f"Среднее количество источников: {np.mean(sources_found):.1f}")
        print(f"Средняя длина ответа: {np.mean(answer_lengths):.0f} символов")

# Тестовые запросы
test_queries = [
    "Что такое Cynosure Bridge?",
    "Как настроить Bridge для использования?",
    "Какие модели поддерживаются?",
    "Поддерживает ли Bridge streaming?",
    "Как работают embeddings в Bridge?",
    "Какой порт использует сервис?"
]

# Запуск бенчмарка
analytics = RAGAnalytics()
analytics.benchmark_search(rag, test_queries)

## 🎯 Заключение

Этот notebook демонстрирует полную реализацию RAG системы с использованием Cynosure Bridge:

- ✅ **Векторное хранилище** с FAISS индексом
- ✅ **Семантический поиск** через embeddings
- ✅ **Контекстная генерация** ответов
- ✅ **История разговора** для мульти-турн диалогов
- ✅ **Бенчмарк производительности**

### 🚀 Следующие шаги:

1. **Добавьте больше документов** в базу знаний
2. **Настройте фильтрацию** по типам документов  
3. **Реализуйте персистентное хранилище** (ChromaDB, Pinecone)
4. **Добавьте веб-интерфейс** для удобного взаимодействия
5. **Настройте мониторинг** и логирование